* load wfstore
* load annots Store
* choose chunker
* choose featurizer
* choose train/test split: mark sessions as train or test


# Using pipeline maker

In [9]:
import os
import soundfile as sf
import pandas as pd
import zipfile
from io import BytesIO
from pathlib import Path

from dol.appendable import add_append_functionality_to_store_cls
from dol import Store
from dol import FilesOfZip, wrap_kvs, filt_iter

from py2store import FilesOfZip
from hear import WavLocalFileStore
from dol import FuncReader



def my_obj_of_data(b):
    return sf.read(BytesIO(b), dtype="float32")[0]

@wrap_kvs(obj_of_data=my_obj_of_data)
@filt_iter(filt=lambda x: not x.startswith("__MACOSX") and x.endswith(".wav"))
class WfZipStore(FilesOfZip):
    """Waveform access. Keys are .wav filenames and values are numpy arrays of int16 waveform."""

    pass


def key_to_ext(k):
    _, ext = os.path.splitext(k)
    if ext.startswith("."):
        ext = ext[1:]
    return ext


def processor_from_ext(ext):
    if ext.startswith("."):
        ext = ext[1:]
    if ext in {"zip"}:
        pass
    elif ext in {"wav"}:
        pass

def is_zip_file(filepath):
    return zipfile.is_zipfile(filepath)

def is_dir(filepath):
    return os.path.isdir(filepath)

def key_maker(name, prefix):
    return f'{prefix}_{name}'

def wf_store_factory(filepath):
    key = key_maker(name = filepath, prefix='wf_store')
    tag = 'wf_store'

    if is_dir(filepath):
        data = WavLocalFileStore(filepath)

         
    elif is_zip_file(filepath):
        data = WfZipStore(filepath)

    return mk_store_item(key, tag, data)

def annot_store_factory(filepath):
    key = key_maker(name = filepath, prefix='annot_store')
    tag = 'annot_store'

    data = pd.read_csv(filepath)

    return mk_store_item(key, tag, data)

def mk_store_item(key, tag, data):
    return dict(key = key, tag = tag, data=data)

def append_to_store(store, item):
    store.append(item)

def dacc_factory():
    pass

factory_store = {'wf_factory': wf_store_factory, 'dacc':None, 'annot_factory':annot_store_factory}

#factory_store = FuncReader([wf_store_factory, dacc_factory])

store_cls = Store
item2kv = lambda item: (item['key'], item['data'])

appendable_store_cls = add_append_functionality_to_store_cls(store_cls, item2kv=item2kv)

In [10]:
# example
rootdir = '/Users/sylvain/Dropbox/Otosense/VacuumEdgeImpulse/'
annot_filepath = '/Users/sylvain/Dropbox/sipyb/Testing/data/annots_vacuum.csv'
#is_dir(rootdir)

In [16]:
# mock functions
def mk_global_store():
    return appendable_store_cls()

def select_wf_factory():
    return {'key':'wf_store_factory', 'data':wf_store_factory}

def select_annot_factory():
    return {'key':'annot_store_factory', 'data':annot_store_factory}

In [17]:
# user story: make a global store and add a wfstore to it

global_store = mk_global_store()
wf_factory = select_wf_factory()
global_store.append(wf_factory)
annot_factory = select_annot_factory()
global_store.append(annot_factory)


In [18]:
global_store

{'wf_store_factory': <function wf_store_factory at 0x7fcf00737400>, 'annot_store_factory': <function annot_store_factory at 0x7fcf01494160>}

# Without using pipeline maker

* load wfstore
* load annots Store
* choose chunker
* choose featurizer
* choose model
* train model


In [21]:
d = dict(key = 'example')

In [22]:
d()

TypeError: 'dict' object is not callable